[View in Colaboratory](https://colab.research.google.com/github/ebgv/Planet--Understanding-the-Amazon-from-Space/blob/master/model2_hidden_layers.ipynb)

# Imports 

In [1]:
# to install pytorch on colab
from os import path
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())

accelerator = 'cu80' if path.exists('/opt/bin/nvidia-smi') else 'cpu'

!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.4.1-{platform}-linux_x86_64.whl torchvision

tcmalloc: large alloc 1073750016 bytes == 0x55e1e000 @  0x7fdf5cfb22a4 0x594e17 0x626104 0x51190a 0x4f5277 0x510c78 0x5119bd 0x4f5277 0x4f3338 0x510fb0 0x5119bd 0x4f5277 0x4f3338 0x510fb0 0x5119bd 0x4f5277 0x4f3338 0x510fb0 0x5119bd 0x4f6070 0x510c78 0x5119bd 0x4f5277 0x4f3338 0x510fb0 0x5119bd 0x4f6070 0x4f3338 0x510fb0 0x5119bd 0x4f6070


In [2]:
!pip install -U bcolz

    100% |████████████████████████████████| 1.5MB 9.0MB/s 
  Running setup.py bdist_wheel for bcolz ... - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ done
  Stored in directory: /root/.cache/pip/wheels/9f/78/26/fb8c0acb91a100dc8914bf236c4eaa4b207cb876893c40b745
Successfully built bcolz


In [3]:
!pip install Pillow==4.0.0

    100% |████████████████████████████████| 5.6MB 7.4MB/s 
torchvision 0.2.1 has requirement pillow>=4.1.1, but you'll have pillow 4.0.0 which is incompatible.
  Found existing installation: Pillow 5.3.0
    Uninstalling Pillow-5.3.0:
      Successfully uninstalled Pillow-5.3.0


In [0]:
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import os
import torch
import torch.nn as nn
import torchvision
from torchvision import models,transforms,datasets
import bcolz
import time

# GPU Settings 

In [5]:
use_gpu = torch.cuda.is_available()
print('Using gpu: %s ' % use_gpu)

dtype = torch.FloatTensor
if use_gpu:
    dtype = torch.cuda.FloatTensor

Using gpu: True 


# Data Loader

In [0]:
# loader taken from class example: parameters to verify

def save_array(fname, arr):
    c=bcolz.carray(arr, rootdir=fname, mode='w')
    c.flush()
def load_array(fname):
    return bcolz.open(fname)[:]

normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]) # statistics from imagenet ? 

prep1 = transforms.Compose([
                transforms.CenterCrop(224), # default cropping 
                transforms.ToTensor(),
                normalize,
            ])

In [0]:
data_dir = '/content/data/sample'
#data_dir = '/content/data'

#batch_size = 4
batch_size = 64

In [0]:
# dictionary of datasets - in this case only train data set 
dsets = {x: datasets.ImageFolder(os.path.join(data_dir, x), prep1)
         for x in ['train', 'test']}

In [9]:
# train labels 
print(dsets['train'].imgs[:5]) # 5 first images and labels 
print(dsets['train'].imgs[len(dsets['train'])-5:]) # 5 last images and labels 

[('/content/data/sample/train/clear/train_10031.jpg', 0), ('/content/data/sample/train/clear/train_10042.jpg', 0), ('/content/data/sample/train/clear/train_10061.jpg', 0), ('/content/data/sample/train/clear/train_10085.jpg', 0), ('/content/data/sample/train/clear/train_10143.jpg', 0)]
[('/content/data/sample/train/partly_cloudy/train_9892.jpg', 3), ('/content/data/sample/train/partly_cloudy/train_9899.jpg', 3), ('/content/data/sample/train/partly_cloudy/train_9913.jpg', 3), ('/content/data/sample/train/partly_cloudy/train_9974.jpg', 3), ('/content/data/sample/train/partly_cloudy/train_9976.jpg', 3)]


In [10]:
# test labels 
print(dsets['test'].imgs[:5]) # 5 first images and labels 
print(dsets['test'].imgs[len(dsets['test'])-5:]) # 5 last images and labels 

[('/content/data/sample/test/clear/train_10046.jpg', 0), ('/content/data/sample/test/clear/train_10136.jpg', 0), ('/content/data/sample/test/clear/train_10156.jpg', 0), ('/content/data/sample/test/clear/train_10185.jpg', 0), ('/content/data/sample/test/clear/train_1042.jpg', 0)]
[('/content/data/sample/test/partly_cloudy/train_9483.jpg', 3), ('/content/data/sample/test/partly_cloudy/train_9520.jpg', 3), ('/content/data/sample/test/partly_cloudy/train_9660.jpg', 3), ('/content/data/sample/test/partly_cloudy/train_9802.jpg', 3), ('/content/data/sample/test/partly_cloudy/train_9979.jpg', 3)]


In [11]:
dset_classes = dsets['train'].classes
dset_classes # binary classes to begin 

['clear', 'cloudy', 'haze', 'partly_cloudy']

In [0]:
# dictionary of data loaders - again only train for now 
dset_loaders = {x: torch.utils.data.DataLoader(dsets[x], batch_size=batch_size,
                                               shuffle=True, num_workers=0)
                for x in ['train', 'test']}

In [13]:
print(len(dsets['train']))
print(len(dsets['test']))

6400
1600


# Model: A bit more complex one

In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class NNet(nn.Module):
    def __init__(self):
        super(NNet, self).__init__()
        self.conv1 = nn.Conv2d(in_channels = 3, out_channels = 32, kernel_size=5)
        self.conv2 = nn.Conv2d(in_channels = 32, out_channels = 64, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(in_features = 4096, out_features = 256)
        self.fc2 = nn.Linear(in_features = 256, out_features = 4)

    def forward(self, x):
        # First Conv block:
        x = self.conv1(x)
        x = F.max_pool2d(x, kernel_size = 5, stride = 5)
        x = F.relu(x)
        
        # Second Conv block:
        x = self.conv2(x)
        x = self.conv2_drop(x)
        x = F.max_pool2d(x, kernel_size = 5, stride = 5)
        x = F.relu(x)
        
        # Flatten:
        x = x.view(x.size(0), -1)
        
        # Classifier:
        ## First layer:
        x = self.fc1(x)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        # Second layer:
        x = self.fc2(x)
        x = F.log_softmax(x, dim=1)

        return x

# Training Framework

In [0]:
def train(model,data_loader,loss_fn,optimizer,n_epochs=1):
    
    model.train(True)
    
    loss_train = np.zeros(n_epochs)
    acc_train = np.zeros(n_epochs)
    
    for epoch_num in range(n_epochs):
        running_corrects = 0.0
        running_loss = 0.0
        size = 0

        for data in data_loader:
            inputs, labels = data
            if use_gpu:
                inputs, labels = inputs.cuda(), labels.cuda()    
                
            # batch_size ?
            bs = labels.size(0)
            
            # define the loss to minimize
            outputs = model(inputs)
            loss = loss_fn(outputs, labels)
            # define the optimizer
            optimizer = optimizer
            optimizer.zero_grad()
            # backpropagation
            loss.backward()
            optimizer.step()
            
            # predictions to get statistics 
            _,preds = torch.max(outputs.data,1)
            # statistics
            running_loss += loss.data.item()
            running_corrects += torch.sum(preds == labels.data)
            size += bs
        # epoch statistics     
        epoch_loss = running_loss / size
        epoch_acc = running_corrects.item() / size
        loss_train[epoch_num] = epoch_loss
        acc_train[epoch_num] = epoch_acc
        print('Train - Loss: {:.4f} Acc: {:.4f}'.format(epoch_loss, epoch_acc))
        
    return loss_train, acc_train

# Running training epochs

In [16]:
# instanciate the model 
model = NNet()
use_gpu = torch.cuda.is_available()
if use_gpu:
    model = model.cuda()

# choose the appropriate loss
loss_fn = torch.nn.CrossEntropyLoss()
# learning rate 
learning_rate = 1e-3
# your SGD optimizer
optimizer_cl = torch.optim.Adam(model.parameters(), lr=learning_rate)


# and train for 20 epochs
l_t, a_t = train(model, dset_loaders['train'], loss_fn, optimizer_cl, n_epochs = 20)

Train - Loss: 0.0146 Acc: 0.6025
Train - Loss: 0.0130 Acc: 0.6553
Train - Loss: 0.0123 Acc: 0.6852
Train - Loss: 0.0119 Acc: 0.6981
Train - Loss: 0.0112 Acc: 0.7172
Train - Loss: 0.0109 Acc: 0.7297
Train - Loss: 0.0107 Acc: 0.7348
Train - Loss: 0.0105 Acc: 0.7483
Train - Loss: 0.0103 Acc: 0.7544
Train - Loss: 0.0101 Acc: 0.7575
Train - Loss: 0.0098 Acc: 0.7594
Train - Loss: 0.0093 Acc: 0.7739
Train - Loss: 0.0092 Acc: 0.7786
Train - Loss: 0.0090 Acc: 0.7816
Train - Loss: 0.0090 Acc: 0.7792
Train - Loss: 0.0090 Acc: 0.7839
Train - Loss: 0.0089 Acc: 0.7887
Train - Loss: 0.0087 Acc: 0.7861
Train - Loss: 0.0083 Acc: 0.7956
Train - Loss: 0.0081 Acc: 0.8014


# Testing 

In [0]:
def test(model,data_loader):
    model.train(False)

    running_corrects = 0.0
    running_loss = 0.0
    size = 0

    for data in data_loader:
        inputs, labels = data
        if use_gpu:
            inputs, labels = inputs.cuda(), labels.cuda()
            
        bs = labels.size(0)
                
        outputs = model(inputs)
        loss = loss_fn(outputs, labels)
        _,preds = torch.max(outputs.data,1)
        
        # statistics
        running_loss += loss.data.item()
        running_corrects += torch.sum(preds == labels.data)
        size += bs

    print('Test - Loss: {:.4f} Acc: {:.4f}'.format(running_loss / size, running_corrects.item() / size))

In [18]:
# using 'hidden'

test(model, dset_loaders['test'])

Test - Loss: 0.0087 Acc: 0.7844
